# Testing: quick_find_parameters()

In [1]:
from simulation.mean_var_autocorr import check_biological_appropriateness, find_parameters
import numpy as np
import tqdm
%load_ext autoreload
%autoreload 2

# Define a function to try different values
def find_biologically_appropriate_parameters(target_cv_initial=1.7, mu_target_initial=5, max_attempts=10):
    # Try different combinations of CV and mean until we find biologically appropriate values
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            target_cv = target_cv_initial
            mu = mu_target_initial
        # On subsequent attempts, adjust the values
        else:
            # Try reducing CV or increasing mean to improve Fano factor and CV
            target_cv = target_cv_initial * (0.9 - attempt * 0.05)  # Gradually reduce CV
            mu = mu_target_initial * (1.1 + attempt * 0.1)         # Gradually increase mean
        
        # Calculate corresponding variance
        variance = (target_cv * mu)**2
        
        print(f"\nAttempt {attempt+1}/{max_attempts}")
        print(f"Testing: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
        
        # Check if these values are biologically appropriate
        is_appropriate = check_biological_appropriateness(variance, mu)
        
        if is_appropriate:
            print(f"✅ Found biologically appropriate values: CV = {target_cv:.2f}, Mean = {mu:.2f}, Variance = {variance:.2f}")
            return target_cv, mu, variance
    
    # If we reach here, we couldn't find appropriate parameters
    print("❌ Could not find biologically appropriate parameters")
    return None, None, None

# Function to pre-check all CV ratios for biological appropriateness
def find_parameters_without_skipping(initial_cv_normal=1.7, initial_mu_target=5, cv_ratios=np.arange(0.1, 3.0, 0.01), max_attempts=5):
    for attempt in range(max_attempts):
        # On first attempt, use the initial values
        if attempt == 0:
            cv_normal = initial_cv_normal
            mu = initial_mu_target
        # On subsequent attempts, adjust the values
        else:
            # Try decreasing CV and increasing mean to improve all conditions
            cv_normal = initial_cv_normal * (0.85 - attempt * 0.05)  # More aggressive CV reduction
            mu = initial_mu_target * (1.3 + attempt * 0.2)          # More aggressive mean increase
        
        # Calculate normal variance
        variance_normal = (cv_normal * mu)**2
        
        print(f"\nTesting parameters: CV normal = {cv_normal:.2f}, Mean = {mu:.2f}")
        
        # Check if normal condition is biologically appropriate
        is_normal_appropriate = check_biological_appropriateness(variance_normal, mu)
        if not is_normal_appropriate:
            print(f"⚠️ Normal condition not biologically appropriate. Trying different parameters...")
            continue
        
        # Check all CV ratios for the stress condition
        all_appropriate = True
        inappropriate_ratios = []
        
        for ratio in tqdm.tqdm(cv_ratios, desc="Checking all CV ratios"):
            cv_stress = ratio * cv_normal
            variance_stress = (cv_stress * mu)**2
            
            is_stress_appropriate = check_biological_appropriateness(variance_stress, mu)
            if not is_stress_appropriate:
                all_appropriate = False
                inappropriate_ratios.append(ratio)
        
        if all_appropriate:
            print(f"✅ Found parameters where all CV ratios are biologically appropriate!")
            print(f"CV normal = {cv_normal:.2f}, Mean = {mu:.2f}, Variance normal = {variance_normal:.2f}")
            return cv_normal, mu, variance_normal
        else:
            print(f"⚠️ {len(inappropriate_ratios)} out of {len(cv_ratios)} CV ratios are not biologically appropriate")
            print(f"First few inappropriate ratios: {inappropriate_ratios[:5]}")
    
    print("❌ Could not find parameters where all CV ratios are biologically appropriate")
    print("You may need to adjust the range of CV ratios or try more extreme parameter values")
    return None, None, None


## Let's get some biologically sensible parameters firt

In [2]:
cv_ratio = np.arange(0.5, 2.0, 0.01)  

# Set initial parameters for search
initial_cv_normal = 0.32  # Try a lower initial CV for normal condition
initial_mu_target = 46.5   # Try a higher initial mean
autocorr_target = 1.5    # Autocorrelation (same for both conditions)

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
target_cv_normal, mu_target, variance_normal = find_parameters_without_skipping(
    initial_cv_normal=initial_cv_normal, 
    initial_mu_target=initial_mu_target,
    cv_ratios=cv_ratio,
    max_attempts=10  # Increase max attempts to try more parameter combinations
)

# If we couldn't find parameters that work for all ratios, use the original function
if target_cv_normal is None:
    print("\nFalling back to finding biologically appropriate parameters for normal condition only...")
    target_cv_normal, mu_target, variance_normal = find_biologically_appropriate_parameters(
        target_cv_initial=initial_cv_normal, 
        mu_target_initial=initial_mu_target
    )

Finding parameters where all CV ratios are biologically appropriate...

Testing parameters: CV normal = 0.32, Mean = 46.50
✅ System is biologically appropriate with Fano factor: 4.76, CV: 0.32


Checking all CV ratios: 100%|██████████| 150/150 [00:00<00:00, 62707.62it/s]

✅ System is biologically appropriate with Fano factor: 1.19, CV: 0.16
✅ System is biologically appropriate with Fano factor: 1.24, CV: 0.16
✅ System is biologically appropriate with Fano factor: 1.29, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.34, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.39, CV: 0.17
✅ System is biologically appropriate with Fano factor: 1.44, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.49, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.55, CV: 0.18
✅ System is biologically appropriate with Fano factor: 1.60, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.66, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.71, CV: 0.19
✅ System is biologically appropriate with Fano factor: 1.77, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.83, CV: 0.20
✅ System is biologically appropriate with Fano factor: 1.89, CV: 0.20
✅ System is biologic

## Test out the function by trying to find parameters across the CV range

In [3]:
# Test the quick_find_parameters function across CV ratios
import time
import numpy as np
from simulation.mean_cv_autocorr import quick_find_parameters
from simulation.mean_var_autocorr import check_biological_appropriateness

%load_ext autoreload
%autoreload 2

# Define test parameters
sigma_b_test = 46
mu_target_test = 46
autocorr_target_test = 1.5
base_cv = 0.32

# Define a range of CV values to test
cv_ratios = np.arange(0.5, 2.0, 0.01)

results = []

for ratio in tqdm.tqdm(cv_ratios, desc="Testing CV ratios"):
    cv_target_test = ratio * base_cv
    variance_test = (cv_target_test * mu_target_test)**2
    
    print(f"\nTesting CV ratio: {ratio:.2f}, CV: {cv_target_test:.2f}")
    print(f"Mean: {mu_target_test:.2f}, Variance: {variance_test:.2f}")
    
    # Check if biologically appropriate
    is_appropriate = check_biological_appropriateness(variance_test, mu_target_test)
    if not is_appropriate:
        print(f"⚠️ Parameters not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        continue

    # Test quick_find_parameters
    start_time = time.time()
    try:
        quick_result = quick_find_parameters(
            sigma_b=sigma_b_test,
            mu_target=mu_target_test,
            autocorr_target=autocorr_target_test,
            cv_target=cv_target_test
        )
        quick_time = time.time() - start_time
        print(f"quick_find_parameters: rho={quick_result[0]:.4f}, sigma_u={quick_result[1]:.4f}, d={quick_result[2]:.4f}, time={quick_time:.4f}s")
    except Exception as e:
        print(f"Error with quick_find_parameters: {str(e)}")
        quick_time = None
        quick_result = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Testing CV ratios:   0%|          | 0/150 [00:00<?, ?it/s]

/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:36: RuntimeWarning: overflow encountered in exp
  return -1/np.exp(1) + (-(d_val*mu_target*(1 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target))) + (-1 + cv_sq_target*mu_target**2 + cv_sq_target**2*mu_target**2 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target)**2)*sigma_b)/(cv_sq_target*np.exp(autocorr_target*d_val)*mu_target*(-(d_val*mu_target) + (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))
/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:36: RuntimeWarning: invalid value encountered in scalar add
  return -1/np.exp(1) + (-(d_val*mu_target*(1 + np.exp((autocorr_target*d_val*(d_val*mu_target - (


Testing CV ratio: 0.50, CV: 0.16
Mean: 46.00, Variance: 54.17
✅ System is biologically appropriate with Fano factor: 1.18, CV: 0.16
Found valid solution for D=0.6666666666666666: rho=42.00759612399706, sigma_u=16.909692461631355, d=0.6677444202992648
Found solution: rho=42.0076, sigma_u=16.9097, d=0.6677
quick_find_parameters: rho=42.0076, sigma_u=16.9097, d=0.6677, time=0.0264s

Testing CV ratio: 0.51, CV: 0.16
Mean: 46.00, Variance: 56.36
✅ System is biologically appropriate with Fano factor: 1.23, CV: 0.16
Found valid solution for D=0.6666666666666666: rho=46.574861053365574, sigma_u=23.738473157323558, d=0.6678503119547369
Found solution: rho=46.5749, sigma_u=23.7385, d=0.6679
quick_find_parameters: rho=46.5749, sigma_u=23.7385, d=0.6679, time=0.0033s

Testing CV ratio: 0.52, CV: 0.17
Mean: 46.00, Variance: 58.59
✅ System is biologically appropriate with Fano factor: 1.27, CV: 0.17
Found valid solution for D=0.6666666666666666: rho=52.36378858280144, sigma_u=32.40106802347442, d=0

Testing CV ratios: 100%|██████████| 150/150 [00:00<00:00, 340.71it/s]


Testing CV ratio: 1.15, CV: 0.37
Mean: 46.00, Variance: 286.56
✅ System is biologically appropriate with Fano factor: 6.23, CV: 0.37
Found valid solution for D=0.6666666666666666: rho=645517750.0165462, sigma_u=123437565.31295398, d=5.2295061703677295
quick_find_parameters: rho=645517750.0165, sigma_u=123437565.3130, d=5.2295, time=0.0032s

Testing CV ratio: 1.16, CV: 0.37
Mean: 46.00, Variance: 291.56
✅ System is biologically appropriate with Fano factor: 6.34, CV: 0.37
Found valid solution for D=0.6666666666666666: rho=511863412.3818254, sigma_u=95884743.71249622, d=5.338317098223939
quick_find_parameters: rho=511863412.3818, sigma_u=95884743.7125, d=5.3383, time=0.0032s

Testing CV ratio: 1.17, CV: 0.37
Mean: 46.00, Variance: 296.61
✅ System is biologically appropriate with Fano factor: 6.45, CV: 0.37
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: -182621730.33802783
Error with quick_fin

``quick_find_parameter()`` seems to be working fine with biologically relevant systems

## Let's try and run this for 2 conditions

In [22]:
import tqdm
from simulation.mean_var_autocorr import check_biological_appropriateness 
from simulation.mean_cv_autocorr import quick_find_parameters
###############################################################################
# Generate Synthetic Data
# 1) Define target mean, CV and autocorrelations, and some parameters to start with
###############################################################################

# Set initial parameters based on our testing in the notebook
# Initial parameters
parameters = {
    "stress": {"sigma_b": 1.0},
    "normal": {"sigma_b": 2.0}
}
target_cv_normal = 0.5  # initial CV for normal condition
mu_target = 20   # Try a higher initial mean
autocorr_target = 1    # Autocorrelation time (same for both conditions)

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
# Define a narrower CV ratio range based on testing
cv_ratio = np.arange(0.5, 2.0, 0.01)  # Narrower range that's more likely to be biologically appropriate

print(f"\nStarting simulations with:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Autocorrelation Time: {autocorr_target:.2f}")

###############################################################################
# 2) Loop over different CV ratios
###############################################################################

# Track inappropriate ratios for reporting
inappropriate_ratios = []
param_finding_failures = []  # Track any failures in parameter finding

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2
    variance_normal = (target_cv_normal * mu_target)**2

    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Double-check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)

    # If stress parameters are not biologically appropriate, track it and skip
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        inappropriate_ratios.append(ratio)
        continue
    
    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            # Pass the sigma_b value from the param dictionary, not the whole dictionary
            rho, sigma_u, d = quick_find_parameters(
                param.get('sigma_b'), mu_target=mu_target, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition
            )

            results[condition] = {"rho": rho, "sigma_u": sigma_u, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            param_finding_failures.append({"condition": condition, "sigma_b": param.get('sigma_b'), 'mu_target': mu_target, 'autocorr_target': autocorr_target, 'cv_for_condition': cv_for_condition, "error": str(e)})
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": results["stress"]["sigma_u"], 
        "sigma_b": parameters["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": results["normal"]["sigma_u"], 
        "sigma_b": parameters["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]
    
    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

Finding parameters where all CV ratios are biologically appropriate...

Starting simulations with:
Normal CV: 0.50, Mean: 20.00, Autocorrelation Time: 1.00


Running CV Ratio Simulations:   0%|          | 0/150 [00:00<?, ?it/s]

Running CV Ratio Simulations:  18%|█▊        | 27/150 [00:00<00:00, 262.20it/s]


Testing CV ratio: 0.50, Stress CV: 0.25, Normal CV: 0.50
Corresponding variances - Stress: 25.00, Normal: 100.00
✅ System is biologically appropriate with Fano factor: 1.25, CV: 0.25
Found valid solution for D=1.0: rho=25.493990993880285, sigma_u=0.022757644633867966, d=1.2463358806282379
Found solution: rho=25.4940, sigma_u=0.0228, d=1.2463
[STRESS] ✅ Found: {'rho': 25.493990993880285, 'sigma_u': 0.022757644633867966, 'd': 1.2463358806282379}
Found valid solution for D=1.0: rho=47.01785785627025, sigma_u=1.3610557685291587, d=1.3989014492564003
Found solution: rho=47.0179, sigma_u=1.3611, d=1.3989
[NORMAL] ✅ Found: {'rho': 47.01785785627025, 'sigma_u': 1.3610557685291587, 'd': 1.3989014492564003}
Updated Parameter Sets: [{'sigma_u': 0.022757644633867966, 'sigma_b': 1.0, 'rho': 25.493990993880285, 'd': 1.2463358806282379, 'label': 0}, {'sigma_u': 1.3610557685291587, 'sigma_b': 2.0, 'rho': 47.01785785627025, 'd': 1.3989014492564003, 'label': 1}]

Testing CV ratio: 0.51, Stress CV: 0.26

Running CV Ratio Simulations:  36%|███▌      | 54/150 [00:00<00:00, 266.38it/s]


Testing CV ratio: 1.04, Stress CV: 0.52, Normal CV: 0.50
Corresponding variances - Stress: 108.16, Normal: 100.00
✅ System is biologically appropriate with Fano factor: 5.41, CV: 0.52
Found valid solution for D=1.0: rho=78.49970417074441, sigma_u=0.3179352795662377, d=2.9781319837109317
Found solution: rho=78.4997, sigma_u=0.3179, d=2.9781
[STRESS] ✅ Found: {'rho': 78.49970417074441, 'sigma_u': 0.3179352795662377, 'd': 2.9781319837109317}
Found valid solution for D=1.0: rho=47.01785785627025, sigma_u=1.3610557685291587, d=1.3989014492564003
Found solution: rho=47.0179, sigma_u=1.3611, d=1.3989
[NORMAL] ✅ Found: {'rho': 47.01785785627025, 'sigma_u': 1.3610557685291587, 'd': 1.3989014492564003}
Updated Parameter Sets: [{'sigma_u': 0.3179352795662377, 'sigma_b': 1.0, 'rho': 78.49970417074441, 'd': 2.9781319837109317, 'label': 0}, {'sigma_u': 1.3610557685291587, 'sigma_b': 2.0, 'rho': 47.01785785627025, 'd': 1.3989014492564003, 'label': 1}]

Testing CV ratio: 1.05, Stress CV: 0.53, Normal

Running CV Ratio Simulations:  72%|███████▏  | 108/150 [00:00<00:00, 267.63it/s]

Found solution: rho=82.3145, sigma_u=0.5101, d=2.7255
[STRESS] ✅ Found: {'rho': 82.31452488488242, 'sigma_u': 0.5100799128205783, 'd': 2.72550227925132}
Found valid solution for D=1.0: rho=47.01785785627025, sigma_u=1.3610557685291587, d=1.3989014492564003
Found solution: rho=47.0179, sigma_u=1.3611, d=1.3989
[NORMAL] ✅ Found: {'rho': 47.01785785627025, 'sigma_u': 1.3610557685291587, 'd': 1.3989014492564003}
Updated Parameter Sets: [{'sigma_u': 0.5100799128205783, 'sigma_b': 1.0, 'rho': 82.31452488488242, 'd': 2.72550227925132, 'label': 0}, {'sigma_u': 1.3610557685291587, 'sigma_b': 2.0, 'rho': 47.01785785627025, 'd': 1.3989014492564003, 'label': 1}]

Testing CV ratio: 1.24, Stress CV: 0.62, Normal CV: 0.50
Corresponding variances - Stress: 153.76, Normal: 100.00
✅ System is biologically appropriate with Fano factor: 7.69, CV: 0.62
Found valid solution for D=1.0: rho=82.39201144406599, sigma_u=0.5225788122085289, d=2.7056731245509336
Found solution: rho=82.3920, sigma_u=0.5226, d=2.705

Running CV Ratio Simulations: 100%|██████████| 150/150 [00:00<00:00, 266.87it/s]


Testing CV ratio: 1.85, Stress CV: 0.93, Normal CV: 0.50
Corresponding variances - Stress: 342.25, Normal: 100.00
✅ System is biologically appropriate with Fano factor: 17.11, CV: 0.93
Found valid solution for D=1.0: rho=124.27172890493227, sigma_u=3.8286415666119176, d=1.2868187376364864
Found solution: rho=124.2717, sigma_u=3.8286, d=1.2868
[STRESS] ✅ Found: {'rho': 124.27172890493227, 'sigma_u': 3.8286415666119176, 'd': 1.2868187376364864}
Found valid solution for D=1.0: rho=47.01785785627025, sigma_u=1.3610557685291587, d=1.3989014492564003
Found solution: rho=47.0179, sigma_u=1.3611, d=1.3989
[NORMAL] ✅ Found: {'rho': 47.01785785627025, 'sigma_u': 1.3610557685291587, 'd': 1.3989014492564003}
Updated Parameter Sets: [{'sigma_u': 3.8286415666119176, 'sigma_b': 1.0, 'rho': 124.27172890493227, 'd': 1.2868187376364864, 'label': 0}, {'sigma_u': 1.3610557685291587, 'sigma_b': 2.0, 'rho': 47.01785785627025, 'd': 1.3989014492564003, 'label': 1}]

Testing CV ratio: 1.86, Stress CV: 0.93, N

In [23]:
# size of failures
print(len(param_finding_failures))
print(param_finding_failures)
print(inappropriate_ratios)

0
[]
[]


In [26]:
import tqdm
import numpy as np
from simulation.mean_var_autocorr import check_biological_appropriateness 
from simulation.mean_cv_autocorr import quick_find_parameters

# Define parameter ranges to try
target_cv_normals = [0.10, 0.25, 0.40]
mu_targets = [10, 25, 50]
autocorr_targets = [0.5, 1.5, 5.0]
sigma_b_stress_values = [0.1, 1, 5, 20, 40]
sigma_b_normal_values = [0.1, 1, 5, 15, 30]

# Use a subset of CV ratios for faster testing
test_cv_ratio = np.arange(0.6, 1.8, 0.2)  # Fewer points for quicker testing

# Store results
best_combination = None
min_failures = float('inf')
all_results = []

# Set up progress bar for the outer loop
total_combinations = len(target_cv_normals) * len(mu_targets) * len(autocorr_targets) * len(sigma_b_stress_values) * len(sigma_b_normal_values)
pbar = tqdm.tqdm(total=total_combinations, desc="Testing Parameter Combinations")

for target_cv_normal in target_cv_normals:
    for mu_target in mu_targets:
        for autocorr_target in autocorr_targets:
            for sigma_b_stress in sigma_b_stress_values:
                for sigma_b_normal in sigma_b_normal_values:
                    # Set parameters for this run
                    parameters = {
                        "stress": {"sigma_b": sigma_b_stress},
                        "normal": {"sigma_b": sigma_b_normal}
                    }
                    
                    # Calculate variance for normal condition (needed for printing)
                    variance_normal = (target_cv_normal * mu_target)**2
                    
                    # Reset tracking arrays for this parameter combination
                    inappropriate_ratios = []
                    param_finding_failures = []
                    
                    # Run through CV ratios
                    for ratio in test_cv_ratio:
                        # For the stress condition, we define CV by ratio
                        target_cv_stress = ratio * target_cv_normal
                        
                        # Calculate corresponding variance for stress
                        variance_stress = (target_cv_stress * mu_target)**2
                        
                        # Check if stress parameters are biologically appropriate
                        is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)
                        
                        if not is_stress_appropriate:
                            inappropriate_ratios.append(ratio)
                            continue
                        
                        # Check both conditions
                        success = True
                        
                        for condition, param in parameters.items():
                            # Decide which CV to use for this condition
                            cv_for_condition = target_cv_normal if condition == "normal" else target_cv_stress
                            
                            try:
                                # Fix mean, CV, and autocorrelation
                                rho, sigma_u, d = quick_find_parameters(
                                    param.get('sigma_b'), 
                                    mu_target=mu_target, 
                                    autocorr_target=autocorr_target, 
                                    cv_target=cv_for_condition
                                )
                            except ValueError as e:
                                param_finding_failures.append({
                                    "condition": condition, 
                                    "sigma_b": param.get('sigma_b'), 
                                    'mu_target': mu_target, 
                                    'autocorr_target': autocorr_target, 
                                    'cv_for_condition': cv_for_condition, 
                                    "error": str(e)
                                })
                                success = False
                                break
                        
                        if not success:
                            continue
                    
                    # Store result for this combination
                    result = {
                        "target_cv_normal": target_cv_normal,
                        "mu_target": mu_target,
                        "autocorr_target": autocorr_target,
                        "sigma_b_stress": sigma_b_stress,
                        "sigma_b_normal": sigma_b_normal,
                        "num_failures": len(param_finding_failures),
                        "num_inappropriate_ratios": len(inappropriate_ratios)
                    }
                    all_results.append(result)
                    
                    # Check if this is the best combination so far
                    if len(param_finding_failures) < min_failures:
                        min_failures = len(param_finding_failures)
                        best_combination = {
                            "target_cv_normal": target_cv_normal,
                            "mu_target": mu_target,
                            "autocorr_target": autocorr_target,
                            "sigma_b_stress": sigma_b_stress,
                            "sigma_b_normal": sigma_b_normal,
                            "num_failures": min_failures,
                            "inappropriate_ratios": inappropriate_ratios
                        }
                    
                    # Update progress bar
                    pbar.update(1)
                    
# Close progress bar
pbar.close()



Testing Parameter Combinations:  10%|▉         | 528/5400 [22:26<3:27:07,  2.55s/it]
/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:36: RuntimeWarning: overflow encountered in exp
  return -1/np.exp(1) + (-(d_val*mu_target*(1 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target))) + (-1 + cv_sq_target*mu_target**2 + cv_sq_target**2*mu_target**2 + np.exp((autocorr_target*d_val*(d_val*mu_target - (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))/(d_val*mu_target + sigma_b - cv_sq_target*mu_target*sigma_b))*(-1 + cv_sq_target*mu_target)**2)*sigma_b)/(cv_sq_target*np.exp(autocorr_target*d_val)*mu_target*(-(d_val*mu_target) + (-2 + mu_target + 2*cv_sq_target*mu_target)*sigma_b))
/home/ianyang/stochastic_simulations/src/simulation/mean_cv_autocorr.py:36: RuntimeWarning: invalid value encountered in scalar divide
  return -

⚠️ WARNING: Fano factor 0.04 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789356926, rho: None, sigma_u: -0.00964087106202131
Trying bounded minimization...
res.fun for minimize_scalar: 674.1952363679663
⚠️ WARNING: Fano factor 0.06 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789356643, rho: None, sigma_u: -0.00936084838247701
Trying bounded minimization...
res.fun for minimize_scalar: 205.82115593448293
⚠️ WARNING: Fano factor 0.10 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789356334, rho: None, sigma_u: -0.00900081899264632
Trying bounded minimization...
res.fun for minimize_scalar: 80.44015942325242
⚠️ WARNING: Fano factor 0.14 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789356049, rho: None, sigma_u: -0.008560782719716407
Trying bounded minimization...
res.f

Testing Parameter Combinations:   4%|▍         | 28/675 [00:00<00:04, 133.25it/s]

⚠️ WARNING: Fano factor 0.04 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 467.6696895138357, rho: None, sigma_u: -0.009641862537736925
Trying bounded minimization...
res.fun for minimize_scalar: 564.3573477751186
⚠️ WARNING: Fano factor 0.06 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 467.689595154594, rho: None, sigma_u: -0.00936181396684908
Trying bounded minimization...
res.fun for minimize_scalar: 172.56971134642873
⚠️ WARNING: Fano factor 0.10 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 467.71606611793055, rho: None, sigma_u: -0.009001751028669038
Trying bounded minimization...
res.fun for minimize_scalar: 67.59523796072345
⚠️ WARNING: Fano factor 0.14 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 467.74986807898154, rho: None, sigma_u: -0.008561673356059485
Trying bounded minimization...
re

One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
⚠️ WARNING: Fano factor 0.32 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
⚠️ WARNING: Fano factor 0.04 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
⚠️ WARNING: Fano factor 0.06 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
⚠️ WARNING: Fano factor 0.10 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimi

...

⚠️ WARNING: Fano factor 20.48 is outside the acceptable range (1, 20).
Found valid solution for D=2.0: rho=70025.63753714979, sigma_u=8.010255921200201, d=999.9999676425559
Found valid solution for D=2.0: rho=57000.79680951747, sigma_u=0.014001596022377964, d=999.9999789183389
⚠️ WARNING: Fano factor 25.92 is outside the acceptable range (1, 20).
Found valid solution for D=2.0: rho=75674.31948628918, sigma_u=10.269728663989572, d=999.9999712757948
Found valid solution for D=2.0: rho=57000.79680951747, sigma_u=0.014001596022377964, d=999.9999789183389
✅ System is biologically appropriate with Fano factor: 2.88, CV: 0.24
Found valid solution for D=2.0: rho=169.29429955224572, sigma_u=12.433936250002159, d=2.087866218239045
Found solution: rho=169.2943, sigma_u=12.4339, d=2.0879
Found valid solution for D=2.0: rho=57007.97988639327, sigma_u=0.1401596223505763, d=999.9999784041546
✅ System is biologically appropriate with Fano factor: 5.12, CV: 0.32
Found valid solution for D=2.0: rho=585.

Found solution: rho=82.3181, sigma_u=0.1484, d=1.4336
Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
✅ System is biologically appropriate with Fano factor: 8.00, CV: 0.40
Found valid solution for D=0.6666666666666666: rho=89.9190356180529, sigma_u=0.2644695610823403, d=1.4222412050960793
Found solution: rho=89.9190, sigma_u=0.2645, d=1.4222
Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
✅ System is biologically appropriate with Fano factor: 11.52, CV: 0.48
Found valid solution for D=0.6666666666666666: rho=94.59620093320666, sigma_u=0.4414775941079308, d=1.3124893695173698
Found solution: rho=94.5962, sigma_u=0.4415, d=1.3125
Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
✅ System is biologically appropriate with Fano factor: 15.68, CV: 0.56
Found valid solution for D=0.

Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
✅ System is biologically appropriate with Fano factor: 8.00, CV: 0.40
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: -347273944.0262268
✅ System is biologically appropriate with Fano factor: 11.52, CV: 0.48
Found valid solution for D=0.6666666666666666: rho=1142027587.5578039, sigma_u=108557667.0552525, d=8.416003753480513
Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
✅ System is biologically appropriate with Fano factor: 15.68, CV: 0.56
Found valid solution for D=0.6666666666666666: rho=12631413540.029171, sigma_u=860450421.8819617, d=11.744000706236566
Found valid solution for D=0.6666666666666666: rho=26516.74922547257, sigma_u=0.014003430943781803, d=465.1921263413327
⚠️ WARNING: Fano factor 20.48

Testing Parameter Combinations: 100%|██████████| 675/675 [00:08<00:00, 83.95it/s]

One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
✅ System is biologically appropriate with Fano factor: 8.00, CV: 0.40
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
✅ System is biologically appropriate with Fano factor: 11.52, CV: 0.48
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
✅ System is biologically appropriate with Fano factor: 15.68, CV: 0.56
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for minimize_scalar: nan
⚠️ WARNING: Fano factor 20.48 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: None, rho: None, sigma_u: None
Trying bounded minimization...
res.fun for min

In [27]:
# Sort results by number of failures
sorted_results = sorted(all_results, key=lambda x: (x["num_failures"], x["num_inappropriate_ratios"]))

# Print the top 5 best combinations
print("\nTop 5 Best Parameter Combinations:")
for i, result in enumerate(sorted_results[:5]):
    print(f"{i+1}. CV Normal: {result['target_cv_normal']:.2f}, Mean: {result['mu_target']:.2f}, Autocorr: {result['autocorr_target']:.2f}")
    print(f"   Sigma B (Stress/Normal): {result['sigma_b_stress']}/{result['sigma_b_normal']}")
    print(f"   Failures: {result['num_failures']}, Inappropriate Ratios: {result['num_inappropriate_ratios']}")
    print()

# Print the overall best combination
print("\nBest Parameter Combination:")
print(f"CV Normal: {best_combination['target_cv_normal']:.2f}")
print(f"Mean: {best_combination['mu_target']:.2f}")
print(f"Autocorrelation Target: {best_combination['autocorr_target']:.2f}")
print(f"Sigma B (Stress): {best_combination['sigma_b_stress']}")
print(f"Sigma B (Normal): {best_combination['sigma_b_normal']}")
print(f"Number of Failures: {best_combination['num_failures']}")
print(f"Inappropriate Ratios: {best_combination['inappropriate_ratios']}")


Top 5 Best Parameter Combinations:
1. CV Normal: 0.25, Mean: 50.00, Autocorr: 0.50
   Sigma B (Stress/Normal): 0.1/0.1
   Failures: 0, Inappropriate Ratios: 0

2. CV Normal: 0.25, Mean: 50.00, Autocorr: 0.50
   Sigma B (Stress/Normal): 0.1/1
   Failures: 0, Inappropriate Ratios: 0

3. CV Normal: 0.25, Mean: 50.00, Autocorr: 0.50
   Sigma B (Stress/Normal): 0.1/5
   Failures: 0, Inappropriate Ratios: 0

4. CV Normal: 0.25, Mean: 50.00, Autocorr: 0.50
   Sigma B (Stress/Normal): 0.1/15
   Failures: 0, Inappropriate Ratios: 0

5. CV Normal: 0.25, Mean: 50.00, Autocorr: 0.50
   Sigma B (Stress/Normal): 0.1/30
   Failures: 0, Inappropriate Ratios: 0


Best Parameter Combination:
CV Normal: 0.25
Mean: 50.00
Autocorrelation Target: 0.50
Sigma B (Stress): 0.1
Sigma B (Normal): 0.1
Number of Failures: 0
Inappropriate Ratios: []


In [28]:
# Now run the full simulation with the best parameters
import tqdm
from simulation.mean_var_autocorr import check_biological_appropriateness 
from simulation.mean_cv_autocorr import quick_find_parameters

# Use the best combination found in our grid search
# (Replace these values with the actual best values from the previous cell)
best_params = best_combination  # This will be defined after running the previous cell

# Extract the parameters
target_cv_normal = best_params["target_cv_normal"]
mu_target = best_params["mu_target"]
autocorr_target = best_params["autocorr_target"]
parameters = {
    "stress": {"sigma_b": best_params["sigma_b_stress"]},
    "normal": {"sigma_b": best_params["sigma_b_normal"]}
}

# Define the full CV ratio range for the complete simulation
cv_ratio = np.arange(0.5, 2.0, 0.01)  # Full range for final simulation

print("Running full simulation with optimal parameters:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Autocorrelation Time: {autocorr_target:.2f}")
print(f"Sigma B (Stress): {parameters['stress']['sigma_b']}, Sigma B (Normal): {parameters['normal']['sigma_b']}")

# Calculate variance for normal condition (needed for printing)
variance_normal = (target_cv_normal * mu_target)**2

###############################################################################
# Run simulation with optimal parameters
###############################################################################

# Track inappropriate ratios for reporting
inappropriate_ratios = []
param_finding_failures = []  # Track any failures in parameter finding
successful_parameter_sets = []  # Store successful parameter sets

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2
    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Double-check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)

    # If stress parameters are not biologically appropriate, track it and skip
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        inappropriate_ratios.append(ratio)
        continue
    
    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            rho, sigma_u, d = quick_find_parameters(
                param.get('sigma_b'), mu_target=mu_target, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition
            )

            results[condition] = {"rho": rho, "sigma_u": sigma_u, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            param_finding_failures.append({"condition": condition, "sigma_b": param.get('sigma_b'), 
                                          'mu_target': mu_target, 'autocorr_target': autocorr_target, 
                                          'cv_for_condition': cv_for_condition, "error": str(e), 
                                          "ratio": ratio})
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue

    # Create parameter sets for successful cases
    parameter_set = {
        "cv_ratio": ratio,
        "stress": {
            "sigma_u": results["stress"]["sigma_u"], 
            "sigma_b": parameters["stress"]['sigma_b'], 
            "rho": results["stress"]['rho'], 
            "d": results["stress"]['d'], 
            "label": 0
        },
        "normal": {
            "sigma_u": results["normal"]["sigma_u"], 
            "sigma_b": parameters["normal"]['sigma_b'], 
            "rho": results["normal"]['rho'], 
            "d": results["normal"]['d'], 
            "label": 1
        }
    }
    
    successful_parameter_sets.append(parameter_set)
    
    # Output the results
    print("Updated Parameter Sets for ratio", ratio, ":")
    print(parameter_set)

# Final summary
print("\n--- FINAL SUMMARY ---")
print(f"Total CV ratios tested: {len(cv_ratio)}")
print(f"Biologically inappropriate ratios: {len(inappropriate_ratios)}")
print(f"Parameter finding failures: {len(param_finding_failures)}")
print(f"Successful parameter sets: {len(successful_parameter_sets)}")
print(f"Success rate: {len(successful_parameter_sets)/len(cv_ratio)*100:.2f}%")

Running full simulation with optimal parameters:
Normal CV: 0.25, Mean: 50.00, Autocorrelation Time: 0.50
Sigma B (Stress): 0.1, Sigma B (Normal): 0.1


Running CV Ratio Simulations:  19%|█▉        | 29/150 [00:00<00:00, 284.55it/s]


Testing CV ratio: 0.50, Stress CV: 0.12, Normal CV: 0.25
Corresponding variances - Stress: 39.06, Normal: 156.25
⚠️ WARNING: Fano factor 0.78 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789365945, rho: None, sigma_u: -0.0004375435585756106
Trying bounded minimization...
res.fun for minimize_scalar: 0.4027847930573903
Could not find a valid solution for parameter d
[STRESS] ❌ No suitable solution found.
Found valid solution for D=2.0: rho=52125.22043331494, sigma_u=0.004250443064392365, d=999.9999789184351
[NORMAL] ✅ Found: {'rho': 52125.22043331494, 'sigma_u': 0.004250443064392365, 'd': 999.9999789184351}
Skipping ratio 0.50 due to parameter finding failure

Testing CV ratio: 0.51, Stress CV: 0.13, Normal CV: 0.25
Corresponding variances - Stress: 40.64, Normal: 156.25
⚠️ WARNING: Fano factor 0.81 is outside the acceptable range (1, 20).
One of the following values is None: final d_value: 999.9999789369579, rho: None, sigma_u: -

Running CV Ratio Simulations:  39%|███▊      | 58/150 [00:00<00:00, 265.25it/s]

[STRESS] ✅ Found: {'rho': 52445.567886019606, 'sigma_u': 0.004891137983232519, 'd': 999.9999789191598}
Found valid solution for D=2.0: rho=52125.22043331494, sigma_u=0.004250443064392365, d=999.9999789184351
[NORMAL] ✅ Found: {'rho': 52125.22043331494, 'sigma_u': 0.004250443064392365, 'd': 999.9999789184351}
Updated Parameter Sets for ratio 1.0500000000000005 :
{'cv_ratio': 1.0500000000000005, 'stress': {'sigma_u': 0.004891137983232519, 'sigma_b': 0.1, 'rho': 52445.567886019606, 'd': 999.9999789191598, 'label': 0}, 'normal': {'sigma_u': 0.004250443064392365, 'sigma_b': 0.1, 'rho': 52125.22043331494, 'd': 999.9999789184351, 'label': 1}}

Testing CV ratio: 1.06, Stress CV: 0.27, Normal CV: 0.25
Corresponding variances - Stress: 175.56, Normal: 156.25
✅ System is biologically appropriate with Fano factor: 3.51, CV: 0.27
Found valid solution for D=2.0: rho=52511.51263210206, sigma_u=0.005023027478166635, d=999.9999789192658
[STRESS] ✅ Found: {'rho': 52511.51263210206, 'sigma_u': 0.00502302

...

[STRESS] ✅ Found: {'rho': 52713.0973902939, 'sigma_u': 0.0054261970030221795, 'd': 999.9999789195244}
Found valid solution for D=2.0: rho=52125.22043331494, sigma_u=0.004250443064392365, d=999.9999789184351
[NORMAL] ✅ Found: {'rho': 52125.22043331494, 'sigma_u': 0.004250443064392365, 'd': 999.9999789184351}
Updated Parameter Sets for ratio 1.0900000000000005 :
{'cv_ratio': 1.0900000000000005, 'stress': {'sigma_u': 0.0054261970030221795, 'sigma_b': 0.1, 'rho': 52713.0973902939, 'd': 999.9999789195244, 'label': 0}, 'normal': {'sigma_u': 0.004250443064392365, 'sigma_b': 0.1, 'rho': 52125.22043331494, 'd': 999.9999789184351, 'label': 1}}

Testing CV ratio: 1.10, Stress CV: 0.28, Normal CV: 0.25
Corresponding variances - Stress: 189.06, Normal: 156.25
✅ System is biologically appropriate with Fano factor: 3.78, CV: 0.28
Found valid solution for D=2.0: rho=52781.54248468602, sigma_u=0.005563087194684906, d=999.9999789195929
[STRESS] ✅ Found: {'rho': 52781.54248468602, 'sigma_u': 0.0055630871

Running CV Ratio Simulations:  74%|███████▍  | 111/150 [00:00<00:00, 254.25it/s]

Found valid solution for D=2.0: rho=56901.09657116337, sigma_u=0.013802195541457974, d=999.9999789184099
[STRESS] ✅ Found: {'rho': 56901.09657116337, 'sigma_u': 0.013802195541457974, 'd': 999.9999789184099}
Found valid solution for D=2.0: rho=52125.22043331494, sigma_u=0.004250443064392365, d=999.9999789184351
[NORMAL] ✅ Found: {'rho': 52125.22043331494, 'sigma_u': 0.004250443064392365, 'd': 999.9999789184351}
Updated Parameter Sets for ratio 1.590000000000001 :
{'cv_ratio': 1.590000000000001, 'stress': {'sigma_u': 0.013802195541457974, 'sigma_b': 0.1, 'rho': 56901.09657116337, 'd': 999.9999789184099, 'label': 0}, 'normal': {'sigma_u': 0.004250443064392365, 'sigma_b': 0.1, 'rho': 52125.22043331494, 'd': 999.9999789184351, 'label': 1}}

Testing CV ratio: 1.60, Stress CV: 0.40, Normal CV: 0.25
Corresponding variances - Stress: 400.00, Normal: 156.25
✅ System is biologically appropriate with Fano factor: 8.00, CV: 0.40
Found valid solution for D=2.0: rho=57000.79680951749, sigma_u=0.01400

Running CV Ratio Simulations: 100%|██████████| 150/150 [00:00<00:00, 255.38it/s]

[STRESS] ✅ Found: {'rho': 57303.64824756403, 'sigma_u': 0.01460729891126547, 'd': 999.9999789181193}
Found valid solution for D=2.0: rho=52125.22043331494, sigma_u=0.004250443064392365, d=999.9999789184351
[NORMAL] ✅ Found: {'rho': 52125.22043331494, 'sigma_u': 0.004250443064392365, 'd': 999.9999789184351}
Updated Parameter Sets for ratio 1.630000000000001 :
{'cv_ratio': 1.630000000000001, 'stress': {'sigma_u': 0.01460729891126547, 'sigma_b': 0.1, 'rho': 57303.64824756403, 'd': 999.9999789181193, 'label': 0}, 'normal': {'sigma_u': 0.004250443064392365, 'sigma_b': 0.1, 'rho': 52125.22043331494, 'd': 999.9999789184351, 'label': 1}}

Testing CV ratio: 1.64, Stress CV: 0.41, Normal CV: 0.25
Corresponding variances - Stress: 420.25, Normal: 156.25
✅ System is biologically appropriate with Fano factor: 8.41, CV: 0.41
Found valid solution for D=2.0: rho=57405.84897043151, sigma_u=0.014811700361318217, d=999.999978918044
[STRESS] ✅ Found: {'rho': 57405.84897043151, 'sigma_u': 0.014811700361318

In [29]:
import tqdm
from simulation.mean_var_autocorr import check_biological_appropriateness 
from simulation.mean_var_autocorr import find_parameters
###############################################################################
# Generate Synthetic Data
# 1) Define target mean, CV and autocorrelations, and some parameters to start with
###############################################################################

# Set initial parameters based on our testing in the notebook
# Initial parameters
parameters = {
    "stress": {"sigma_b": 1.0},
    "normal": {"sigma_b": 0.1}
}
target_cv_normal = 0.25  # initial CV for normal condition
mu_target = 50   # Try a higher initial mean
autocorr_target = 0.5    # Autocorrelation time (same for both conditions)

# Try to find parameters that work for all CV ratios
print("Finding parameters where all CV ratios are biologically appropriate...")
# Define a narrower CV ratio range based on testing
cv_ratio = np.arange(0.5, 2.0, 0.01)  # Narrower range that's more likely to be biologically appropriate

print(f"\nStarting simulations with:")
print(f"Normal CV: {target_cv_normal:.2f}, Mean: {mu_target:.2f}, Autocorrelation Time: {autocorr_target:.2f}")

###############################################################################
# 2) Loop over different CV ratios
###############################################################################

# Track inappropriate ratios for reporting
inappropriate_ratios = []
param_finding_failures = []  # Track any failures in parameter finding

for ratio in tqdm.tqdm(cv_ratio, desc="Running CV Ratio Simulations"):
    # For the stress condition, we define CV by ratio
    target_cv_stress = ratio * target_cv_normal
    
    # Calculate corresponding variance values for reporting only
    variance_stress = (target_cv_stress * mu_target)**2
    variance_normal = (target_cv_normal * mu_target)**2

    
    print(f"\nTesting CV ratio: {ratio:.2f}, Stress CV: {target_cv_stress:.2f}, Normal CV: {target_cv_normal:.2f}")
    print(f"Corresponding variances - Stress: {variance_stress:.2f}, Normal: {variance_normal:.2f}")
    
    # Double-check if stress parameters are biologically appropriate
    is_stress_appropriate = check_biological_appropriateness(variance_stress, mu_target)

    # If stress parameters are not biologically appropriate, track it and skip
    if not is_stress_appropriate:
        print(f"⚠️ Stress parameters are not biologically appropriate for ratio {ratio:.2f}. Skipping.")
        inappropriate_ratios.append(ratio)
        continue
    
    # Store results for each condition
    results = {}
    success = True  # Track if both conditions succeed
    
    for condition, param in parameters.items():
        # Decide which CV to use for this condition
        if condition == "normal":
            cv_for_condition = target_cv_normal
        else:  # condition == "stress"
            cv_for_condition = target_cv_stress

        try:
            # Fix mean, CV, and autocorrelation
            # Pass the sigma_b value from the param dictionary, not the whole dictionary
            rho, sigma_u, d = find_parameters(
                param, mu_target=mu_target, autocorr_target=autocorr_target, 
                cv_target=cv_for_condition
            )

            results[condition] = {"rho": rho, "sigma_u": sigma_u, "d": d}
            print(f"[{condition.upper()}] ✅ Found: {results[condition]}")
        except ValueError as e:
            print(f'{e}')
            print(f"[{condition.upper()}] ❌ No suitable solution found.")
            param_finding_failures.append({"condition": condition, "sigma_b": param.get('sigma_b'), 'mu_target': mu_target, 'autocorr_target': autocorr_target, 'cv_for_condition': cv_for_condition, "error": str(e)})
            success = False
    
    # If any condition failed, skip to the next ratio
    if not success:
        print(f"Skipping ratio {ratio:.2f} due to parameter finding failure")
        continue


    # update parameter sets 
    parameter_sets = [
        {"sigma_u": results["stress"]["sigma_u"], 
        "sigma_b": parameters["stress"]['sigma_b'], 
        "rho": results["stress"]['rho'], 
        "d": results["stress"]['d'], 
        "label": 0},
        
        {"sigma_u": results["normal"]["sigma_u"], 
        "sigma_b": parameters["normal"]['sigma_b'], 
        "rho": results["normal"]['rho'], 
        "d": results["normal"]['d'], 
        "label": 1}
    ]
    
    # Output the results
    print("Updated Parameter Sets:", parameter_sets)

Finding parameters where all CV ratios are biologically appropriate...

Starting simulations with:
Normal CV: 0.25, Mean: 50.00, Autocorrelation Time: 0.50


Running CV Ratio Simulations:   0%|          | 0/150 [00:00<?, ?it/s]


Testing CV ratio: 0.50, Stress CV: 0.12, Normal CV: 0.25
Corresponding variances - Stress: 39.06, Normal: 156.25
⚠️ WARNING: Fano factor 0.78 is outside the acceptable range (1, 20).
Attempt 1/10


/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:48: RuntimeWarning: invalid value encountered in multiply
  ACmRNA_eq = sp.exp(-d_val * t) * (
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:49: RuntimeWarning: invalid value encountered in multiply
  d_val * sp.exp((d_val - sigma_u_val - sigma_b) * t) * rho_val * sigma_u_val
/home/ianyang/stochastic_simulations/src/simulation/mean_var_autocorr.py:309: RuntimeWarning: The iteration is not making good progress, as measured by the 
 improvement from the last ten iterations.
  solution = fsolve(
<lambdifygenerated-113>:2: RuntimeWarning: overflow encountered in exp
  return array([[1.0/(d*(sigma_u + 1.0)), -1.0*rho/(d*(sigma_u + 1.0)**2), -1.0*rho/(d**2*(sigma_u + 1.0))], [-sigma_u*(0.606530659712633*d*rho*sigma_u*exp(0.5*d - 0.5*sigma_u) - (sigma_u + 1.0)*(-d**2 + rho*sigma_u + (sigma_u + 1.0)**2))*exp(-0.5*d)/((d - sigma_u - 1.0)*(d*(sigma_u + 1.0) + rho*sigma_u + (sigma_u + 1.0)**2)**2) 

KeyboardInterrupt: 